## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [4]:
import tensorflow as tf

In [5]:
tf.__version__

'1.15.0'

In [0]:
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [8]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import pandas as pd
import numpy as np


In [0]:
data = pd.read_csv('/gdrive/My Drive/External Residency/prices.csv')

### Check all columns in the dataset

In [13]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis=1, inplace=True)

In [15]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_new=data.iloc[0:1000,:]

In [18]:
data_new.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [0]:
y=data_new['volume']/1000


In [23]:
data_new.drop('volume', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(data_new,y, test_size=0.3, random_state=7)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train=np.array(X_train.astype('float32'))
X_test=np.array(X_test.astype('float32'))
y_train=np.array(y_train.astype('float32'))
y_test=np.array(y_test.astype('float32'))

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer


In [0]:
transformer=Normalizer().fit(X_train)

In [0]:
X_train_transform=transformer.transform(X_train)

In [0]:
X_test_transform=transformer.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [150]:
X_train_transform.shape

(700, 4)

In [0]:
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

In [152]:
w

<tf.Tensor: id=15702497, shape=(4, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [153]:
b

<tf.Tensor: id=15702500, shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [156]:
for i in range(100):
   w, b = train(X_train_transform, y_train, w, b, learning_rate=0.01)
   print('Current Loss on iteration', i, loss(y_train, prediction(X_train_transform, w, b)).numpy())
   

Current Loss on iteration 0 273954140.0
Current Loss on iteration 1 273954140.0
Current Loss on iteration 2 273954140.0
Current Loss on iteration 3 273954140.0
Current Loss on iteration 4 273954140.0
Current Loss on iteration 5 273954140.0
Current Loss on iteration 6 273954140.0
Current Loss on iteration 7 273954180.0
Current Loss on iteration 8 273954140.0
Current Loss on iteration 9 273954140.0
Current Loss on iteration 10 273954140.0
Current Loss on iteration 11 273954140.0
Current Loss on iteration 12 273954140.0
Current Loss on iteration 13 273954140.0
Current Loss on iteration 14 273954140.0
Current Loss on iteration 15 273954140.0
Current Loss on iteration 16 273954140.0
Current Loss on iteration 17 273954140.0
Current Loss on iteration 18 273954140.0
Current Loss on iteration 19 273954140.0
Current Loss on iteration 20 273954140.0
Current Loss on iteration 21 273954140.0
Current Loss on iteration 22 273954140.0
Current Loss on iteration 23 273954140.0
Current Loss on iteration 

In [158]:
for i in range(100):
  print('Current Loss on iteration', i*5, loss(y_train, prediction(X_train_transform, w, b)).numpy())

Current Loss on iteration 0 273954140.0
Current Loss on iteration 5 273954140.0
Current Loss on iteration 10 273954140.0
Current Loss on iteration 15 273954140.0
Current Loss on iteration 20 273954140.0
Current Loss on iteration 25 273954140.0
Current Loss on iteration 30 273954140.0
Current Loss on iteration 35 273954140.0
Current Loss on iteration 40 273954140.0
Current Loss on iteration 45 273954140.0
Current Loss on iteration 50 273954140.0
Current Loss on iteration 55 273954140.0
Current Loss on iteration 60 273954140.0
Current Loss on iteration 65 273954140.0
Current Loss on iteration 70 273954140.0
Current Loss on iteration 75 273954140.0
Current Loss on iteration 80 273954140.0
Current Loss on iteration 85 273954140.0
Current Loss on iteration 90 273954140.0
Current Loss on iteration 95 273954140.0
Current Loss on iteration 100 273954140.0
Current Loss on iteration 105 273954140.0
Current Loss on iteration 110 273954140.0
Current Loss on iteration 115 273954140.0
Current Loss o

### Get the shapes and values of W and b

In [159]:
w.shape

TensorShape([Dimension(4), Dimension(1)])

In [160]:
b.shape

TensorShape([Dimension(1)])

### Model Prediction on 1st Examples in Test Dataset

In [177]:
X_test_transform[0:1]

array([[0.5063693 , 0.49396917, 0.4904527 , 0.50896037]], dtype=float32)

In [0]:
res=prediction(X_test_transform[0:1], w, b)

In [180]:
res

<tf.Tensor: id=15718227, shape=(1, 1), dtype=float32, numpy=array([[5610.1084]], dtype=float32)>

In [181]:
y_test[0]

4899.4

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_dataset=pd.read_csv('/gdrive/My Drive/External Residency/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [201]:
iris_dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
X=iris_dataset.iloc[:,1:5]

In [217]:
X

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [0]:
specs=iris_dataset['Species'].str.get_dummies(" ")

In [231]:
specs

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


### Splitting the data into feature set and target set

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,specs, test_size=0.3, random_state=7)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [226]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=200)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 105 samples, validate on 45 samples
Epoch 1/200
105/105 [==============================] - 0s 2ms/sample - loss: 1.8701 - acc: 0.3143 - val_loss: 1.3279 - val_acc: 0.3778
Epoch 2/200
105/105 [==============================] - 0s 173us/sample - loss: 1.4978 - acc: 0.3143 - val_loss: 1.1946 - val_acc: 0.3778
Epoch 3/200
105/105 [==============================] - 0s 175us/sample - loss: 1.3598 - acc: 0.3143 - val_loss: 1.1269 - val_acc: 0.3778
Epoch 4/200
105/105 [==============================] - 0s 152us/sample - loss: 1.2550 - acc: 0.3143 - val_loss: 1.0580 - val_acc: 0.3778
Epoch 5/200
105/105 [==============================] - 0s 162us/sample - loss: 1.1724 - acc: 0.3143 - val_loss: 1.0164 - val_acc: 0.3778
Epoch 6/200
105/105 [==============================] - 0s 164us/sample - loss: 1.1175 - acc: 0.3143 - val_loss: 0.9865 - val_acc: 0.3778
Epoch 7/200
105/105 [====================

### Model Prediction

In [242]:
predictions=model.predict(X_test)

In [246]:
predictions[0]

array([0.00320933, 0.28468776, 0.7121029 ], dtype=float32)

In [249]:
for i in predictions:
  predicted_num = np.argmax([i])
  print(i,predicted_num)

[0.00320933 0.28468776 0.7121029 ] 2
[0.01322371 0.38933855 0.59743774] 2
[9.8722267e-01 1.2181391e-02 5.9593638e-04] 0
[0.01718896 0.40990123 0.57290983] 2
[0.0029998 0.2801375 0.7168627] 2
[0.9601274  0.03688769 0.00298486] 0
[0.04040722 0.4747187  0.4848741 ] 2
[0.04275547 0.47866774 0.47857684] 1
[0.97641945 0.02216039 0.00142015] 0
[0.05319173 0.49320418 0.453604  ] 1
[0.01508103 0.39963642 0.58528256] 2
[0.02931153 0.45120975 0.5194787 ] 2
[9.9253064e-01 7.1914340e-03 2.7789580e-04] 0
[4.8137308e-04 1.7587319e-01 8.2364541e-01] 2
[9.8922008e-01 1.0311778e-02 4.6815249e-04] 0
[0.00732308 0.34384575 0.6488312 ] 2
[0.00134068 0.22967274 0.7689866 ] 2
[0.0026864 0.2728045 0.7245091] 2
[9.8600626e-01 1.3315807e-02 6.7801628e-04] 0
[9.8287040e-01 1.6226528e-02 9.0312975e-04] 0
[0.1201594 0.5295455 0.3502951] 1
[0.00103885 0.21519408 0.78376704] 2
[0.02502273 0.4391485  0.5358288 ] 2
[0.0056777  0.32494718 0.6693751 ] 2
[0.00611804 0.3304402  0.6634418 ] 2
[0.00735995 0.34422374 0.64841

### Save the Model

In [0]:
model.save('iris.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?